In [2]:
from sage.coding.all import *
from sage.coding.grs_code import ReedSolomonCode
from sage.coding.guruswami_sudan.gs_decoder import GRSGuruswamiSudanDecoder
from sage.rings.polynomial.all import *
from sage.rings.finite_rings.all import *
from sage.modules.free_module import VectorSpace
from sage.matrix.all import *
from sage.misc.functional import sqrt
from sage.functions.other import floor
from copy import deepcopy
from sage.misc.prandom import random

In [3]:
P = 15*(2**27) + 1
Fq = GF(P);
R = PolynomialRing(Fq, "X");
X = R.gen()

π = Fq.multiplicative_generator() # Multiplicative Generator 3

omega = π**(15*2**(27 - 10));
omega_order = omega.multiplicative_order()

k = omega_order // 4

RS = ReedSolomonCode(Fq, omega_order, k, primitive_root=omega)
evaluation_points = RS.evaluation_points(); # print(f"Evaluation points: {evaluation_points}")
minimum_distance = RS.minimum_distance(); print(f"Minimum Distance: {minimum_distance}")
dimension = RS.dimension(); print(f"Dimension: {dimension}")
rate = RS.rate(); print(f"Rate: {rate}")
E = RS.encoder("EvaluationPolynomial", polynomial_ring=R)

Minimum Distance: 769
Dimension: 256
Rate: 1/4


In [4]:
def enumerate_points_in_radius(base_point, hamming_distance):
    from itertools import combinations
    base_len = len(base_point)
    result = list()
    for r in range(hamming_distance):
        for entries in combinations(range(base_len), r+1):
            value = list(base_point)
            for j in entries:
                value[j] = 0
            result.append(value)
    return result

In [5]:
def berlekamp_welch(code : ReedSolomonCode, received_vector : VectorSpace):
    n = code.length()
    dim = code.dimension()
    d = code.minimum_distance()
    field = code.base_ring()
    eval_points = code.evaluation_points()
    correctable = d // 2

    if len(received_vector) != n:
        raise ValueError(f"Invalid received vector")
    
    row_entry = lambda eval_point, y_val : [ eval_point**i for i in range(dim+correctable)] + [field(-1)*y_val*(eval_point**i) for i in range(correctable)]

    m = Matrix(field, [ row_entry(x,y) for (x,y) in zip(eval_points, received_vector)])
    # print(m)
    echelon = m.echelon_form()
    # print(echelon)
    if echelon.rank() == n:
        raise ValueError("Failed to decode: Possibly too many errors")
    
    result_vector = field(-1)*echelon.column(echelon.rank())
    a_poly = R(list(result_vector[0:dim+correctable]))
    b_poly = result_vector[dim+correctable:]
    b_poly[echelon.rank() - dim - correctable] = field(1)
    b_poly = R(list(b_poly))
    # print(a_poly)
    # print(b_poly)
    if a_poly % b_poly == 0:
        decoded = a_poly // b_poly
        # print(factor(b_poly))
        return decoded
    else:
        raise ValueError("failed to decode!")


In [ ]:
fx = R.random_element(k-1); print(fx)
code = E.encode(fx)
error_rate = 1 - sqrt(rate)
error_count = floor(error_rate*omega_order) - 1
D = GRSGuruswamiSudanDecoder(RS, tau=error_count)

corrupted_code = deepcopy(code)

for i in range(omega_order):
    if error_count > 0:
        if random() < error_rate:
            corrupted_code[i] = Fq.random_element()
            error_count = error_count - 1
    else:
        break

print(D.decode_to_message(corrupted_code))


1169339104*X^255 + 931763717*X^254 + 1835275699*X^253 + 1528571559*X^252 + 1107217716*X^251 + 964480808*X^250 + 954925453*X^249 + 359737792*X^248 + 1742997006*X^247 + 673845184*X^246 + 618490826*X^245 + 1443625978*X^244 + 287846730*X^243 + 1506573366*X^242 + 1752752857*X^241 + 670722935*X^240 + 1659941995*X^239 + 963152970*X^238 + 987002219*X^237 + 1413088959*X^236 + 1965286363*X^235 + 271093698*X^234 + 1500807421*X^233 + 1388690128*X^232 + 1856045889*X^231 + 529811918*X^230 + 552152259*X^229 + 781487780*X^228 + 1862358691*X^227 + 942284273*X^226 + 1380034917*X^225 + 1841881182*X^224 + 76370325*X^223 + 1256256001*X^222 + 300577916*X^221 + 732406845*X^220 + 1723482055*X^219 + 564533936*X^218 + 1542299664*X^217 + 1459896571*X^216 + 1200875034*X^215 + 659618305*X^214 + 724505432*X^213 + 754210428*X^212 + 412401008*X^211 + 1832418863*X^210 + 129841942*X^209 + 138179551*X^208 + 1990699007*X^207 + 820296670*X^206 + 1459555201*X^205 + 1492498628*X^204 + 1061633934*X^203 + 1518602476*X^202 + 2

KeyboardInterrupt: 